라이브러리 및 데이터 불러오기

In [1]:
import pandas as pd

In [8]:
x_train = pd.read_csv('./book/titanic_x_train.csv',encoding = 'cp949')
y_train = pd.read_csv('./book/titanic_y_train.csv')
x_test = pd.read_csv('./book/titanic_x_test.csv',encoding = 'cp949')

In [9]:
print(x_train.shape,y_train.shape,x_test.shape)

(891, 11) (891, 2) (418, 11)


In [10]:
print(x_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   티켓등급         891 non-null    int64  
 2   승객이름         891 non-null    object 
 3   성별           891 non-null    object 
 4   나이           714 non-null    float64
 5   형제자매배우자수     891 non-null    int64  
 6   부모자식수        891 non-null    int64  
 7   티켓번호         891 non-null    object 
 8   운임요금         891 non-null    float64
 9   객실번호         204 non-null    object 
 10  선착장          889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB
None


In [11]:
print(x_train.isnull().sum())
print(x_train.shape)

PassengerId      0
티켓등급             0
승객이름             0
성별               0
나이             177
형제자매배우자수         0
부모자식수            0
티켓번호             0
운임요금             0
객실번호           687
선착장              2
dtype: int64
(891, 11)


In [12]:
x_train.select_dtypes('object')

,승객이름,성별,티켓번호,객실번호,선착장
0,"Braund, Mr. Owen Harris",male,A/5 21171,NaN,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,NaN,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,NaN,S
...,...,...,...,...,...
886,"Montvila, Rev. Juozas",male,211536,NaN,S
887,"Graham, Miss. Margaret Edith",female,112053,B42,S
888,"Johnston, Miss. Catherine Helen ""Carrie""",female,W./C. 6607,NaN,S
889,"Behr, Mr. Karl Howell",male,111369,C148,C


In [13]:
# 불필요컬럼 제거
x_test_passenger_id = x_test['PassengerId']

x_train = x_train.drop(columns=['PassengerId'])
x_test = x_test.drop(columns=['PassengerId'])
y_train = y_train.drop(columns=['PassengerId'])

x_train = x_train.drop(columns=['티켓번호'])
x_test = x_test.drop(columns=['티켓번호'])

x_train = x_train.drop(columns=['승객이름'])
x_test = x_test.drop(columns=['승객이름'])

x_train = x_train.drop(columns=['나이'])
x_test = x_test.drop(columns=['나이'])

x_train = x_train.drop(columns=['객실번호'])
x_test = x_test.drop(columns=['객실번호'])

In [14]:
# 결측치 처리
x_train['선착장'] = x_train['선착장'].fillna('S')
x_test['선착장'] = x_test['선착장'].fillna('S')

In [15]:
#인코딩
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

x_train['성별'] = x_train['성별'].replace('male',0).replace('female',0)
x_test['성별'] = x_test['성별'].replace('male',0).replace('female',0)
선착장_dummy = pd.get_dummies(x_train['선착장'],drop_first = True).rename(columns={'Q':'선착장Q','S':'선착장S'})
x_train = pd.concat([x_train,선착장_dummy],axis=1)
x_train = x_train.drop(columns = '선착장')

선착장_dummy = pd.get_dummies(x_test['선착장'],drop_first = True).rename(columns={'Q':'선착장Q','S':'선착장S'})
x_test = pd.concat([x_test,선착장_dummy],axis=1)
x_test = x_test.drop(columns = '선착장')

In [16]:
x_train['가족수'] = x_train['형제자매배우자수'] + x_train['부모자식수']
x_test['가족수'] = x_test['형제자매배우자수'] + x_test['부모자식수']

x_train = x_train.drop(columns = ['형제자매배우자수','부모자식수'])
x_test = x_test.drop(columns = ['형제자매배우자수','부모자식수'])

In [17]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [18]:
X_TRAIN, X_TEST, Y_TRAIN,Y_TEST = train_test_split(x_train,y_train,test_size = 0.2, random_state =10)

In [19]:
model = XGBClassifier(n_estimators = 100, max_depth = 5, eval_metric='error',random_state=10)
model.fit(X_TRAIN,Y_TRAIN)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='error', feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=10, ...)

In [23]:
y_test_predicted = pd.DataFrame(model.predict(x_test)).rename(columns = {0:'Survived'})
y_test_predicted

,Survived
0,1
1,0
2,1
3,0
4,1
...,...
413,0
414,1
415,0
416,0


In [24]:
final = pd.concat([x_test_passenger_id,y_test_predicted],axis=1)
final

,PassengerId,Survived
0,892,1
1,893,0
2,894,1
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
